In [30]:
# Import libraries

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import glob
from glob import iglob
import os
import re
import shapely
from shapely.geometry import shape, mapping, Polygon, Point, MultiPolygon
import shapefile
import csv
from csv import DictReader
from collections import OrderedDict
import pprint as pp

In [31]:
# This tells the computer where to find the data. Put it in whatever folder (data) you like.

filepath = ("./data-22/*.csv")

In [32]:
# Combine all years of Citizens Service Bureau complaint files into one file. Here, we'll use 2010-2020.
# It's often more informative, and necessary, to do a year-by-year analysis, and change .csv filenames as necessary.

files = glob.glob(filepath)

header_saved = False
with open('csb-1110-62722-combined.csv','w') as fout:
    for filename in files:
        with open(filename, encoding='cp1252') as fin:
            header = next(fin)
            if not header_saved:
                fout.write(header)
                header_saved = True
            for line in fin:
                fout.write(line)

In [33]:
# Define data types for each column.

dtypes={"CALLERTYPE": object, "DATECANCELLED": object, "DATEINVTDONE": object, "DATETIMECLOSED": object, "DATETIMEINIT": object, "DESCRIPTION": object, "EXPLANATION": object, "GROUP": object, "NEIGHBORHOOD": object, "PLAIN_ENGLISH_NAME_FOR_PROBLEMCODE": object, "PRJCOMPLETEDATE": object, "PROBADDRESS": object, "PROBADDTYPE": object, "PROBCITY": object, "PROBLEMCODE": object, "PROBZIP": object, "REQUESTID": object, "SRX": object, "SRY": object, "STATUS": object, "SUBMITTO": object, "WARD": object}

In [34]:
# Read the CSB combined complaint file you just created into a Pandas dataframe.

data = pd.read_csv('csb-1110-62722-combined.csv', dtype=dtypes)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002320 entries, 0 to 1002319
Data columns (total 22 columns):
 #   Column                              Non-Null Count    Dtype 
---  ------                              --------------    ----- 
 0   CALLERTYPE                          764045 non-null   object
 1   CITY                                155951 non-null   object
 2   DATECANCELLED                       28148 non-null    object
 3   DATEINVTDONE                        955561 non-null   object
 4   DATETIMECLOSED                      946913 non-null   object
 5   DATETIMEINIT                        1002320 non-null  object
 6   DESCRIPTION                         1002311 non-null  object
 7   EXPLANATION                         348638 non-null   object
 8   GROUP                               1002319 non-null  object
 9   NEIGHBORHOOD                        957531 non-null   object
 10  PLAIN_ENGLISH_NAME_FOR_PROBLEMCODE  965568 non-null   object
 11  PRJCOMPLETEDATE         

In [36]:
data.describe()

,CALLERTYPE,CITY,DATECANCELLED,DATEINVTDONE,DATETIMECLOSED,DATETIMEINIT,DESCRIPTION,EXPLANATION,GROUP,NEIGHBORHOOD,...,PROBADDRESS,PROBADDTYPE,PROBLEMCODE,PROBZIP,REQUESTID,SRX,SRY,STATUS,SUBMITTO,WARD
count,764045,155951,28148,955561,946913,1002320,1002311,348638,1002319,957531,...,1001304,1002320,1002320,358107,1002320,984360,984366,1002206,1002009,962664
unique,24,57,26114,848456,920188,986295,461,42,39,123,...,224285,2,384,56,988572,347382,353516,24,83,73
top,PHONE,St. Louis,2020-07-27 07:47:03.0,2014-01-10 12:00:00.0,2019-05-02 07:59:00.0,2013-06-27 15:57:53.0,Inspect City Tree,"The city maintains trees located in parks, med...",Trash/Debris,16,...,1520 MARKET ST,B,Inspect City Tree,63116,529402,904955.77000000,1017648.21000000,CLOSED,"FORESTRY,",27
freq,532292,155799,99,885,1694,40,46535,46535,234806,43637,...,842,866809,46535,44992,24,708,778,942113,197952,45452


In [37]:
# In places where the city isn't listed, insert St. Louis, for processing purposes.

data.CITY = data.CITY.fillna('St. Louis')

In [38]:
# Filling blank spaces with "NA" for not available, for processing purposes.

data.PROBZIP = data.PROBZIP.fillna('NA') 
data.DATETIMECLOSED = data.DATETIMECLOSED.fillna('NA')
data.DATECANCELLED = data.DATECANCELLED.fillna('NA')
data.CALLERTYPE = data.CALLERTYPE.fillna('NA')
data.DATEINVTDONE = data.DATEINVTDONE.fillna('NA')
data.NEIGHBORHOOD = data.NEIGHBORHOOD.fillna('NA')
data.WARD = data.WARD.fillna('NA')

In [39]:
# Setting a variable for coordinate pairs.

loc = ["SRX", "SRY"]

In [40]:
# Dropping data where location values are not available. If we don't have a location, we can't analyze it by location.

data.dropna(axis=0, subset=loc, inplace=True)

In [41]:
# Some of these text values have some trailing or leading whitespace. Here, we trim that away, for processing.

df_obj = data.select_dtypes(['object'])
data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [42]:
# Standardizing the format and spelling of the city.

data.replace("ST LOUIS", "St. Louis", inplace=True)

data.replace("SAINT LOUIS", "St. Louis", inplace=True)

data.replace("Saint Louis", "St. Louis", inplace=True)

data.replace("ST.LOUIS", "St. Louis", inplace=True)

data.replace("St.louis", "St. Louis", inplace=True)

data.replace("ST. LOUIS", "St. Louis", inplace=True)

In [43]:
# Create a variable for the trash codes, or types of complaints, counted in the Post-Dispatch anaylsis.

codes = ["I witnessed someone illegally dumping", "Trash container is overflowing", "Commercial dumpster is overflowing", "Debris dumped in the street or alley", "Debris dumped on an occupied property", "Debris around a vacant building", "Debris on a vacant lot", "Resident improperly disposing of refuse", "Private hauler residential dumpster overflowing", "Raw garbage on property"]

In [44]:
# Filter the data to look at only those codes.

data = data.loc[data['PLAIN_ENGLISH_NAME_FOR_PROBLEMCODE'].isin(codes)]

In [45]:
# Making a column of coordinate pairs so we can check which neighborhood a complaint is in when we have location data, 
# but not a neighborhood number.

data['COORDINATE'] = filtered_data.SRX + ', ' + filtered_data.SRY

data['COORDINATE'] = filtered_data.COORDINATE.str.replace('.00000000' , '')

/Users/jayohday/.virtualenvs/trash/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [47]:
data.to_csv('csb-1110-62722-combined-coords.csv')

In [49]:
# This command runs a separate Python script which determines which neighborhood a complaint came from,
# if a neighborhood number was NA for a complaint; and is based on the X and Y coordinate with the complaint.
# This script, particularly if using all 10 years of data, takes some time to run. It checks each point, and in some
# cases, checks the accuracy of already-ssigned neighborhood numbers. 

%run coords.py

done


In [50]:
# Read in the new .csv the Python script just created, with all neighborhoods filled in, into a Pandas dataframe.

trash = pd.read_csv('csb-1110-62722-combined-cleaned-nhoods.csv')

/Users/jayohday/.virtualenvs/trash/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (10,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [51]:
# There are several hundred records where no neighborhood number was found by coords.py, 
# but a neighborhood number was listed by the city. The provided neighborhood numbers are used, here.

trash['nhood'] = trash['nhood'].fillna(trash['NEIGHBORHOOD'])

In [52]:
# Dropping any records for which, despite our best attempts, we do not have a neighborhood assigned.

trash = trash[trash.nhood != '0']

In [53]:
# Filling any remaining blank spaces in the "nhood" column with "NA" and dropping them.

trash.nhood = trash.nhood.fillna('NA')
trash = trash[trash.nhood != 'NA']

In [54]:
# For spot-checking verification purposes, coords.py ran such that the new "nhood" column used neighborhood numbers 
# and names, and park names. This dictionary is used to map a column with neighborhood and park numbers. 
# (This has to be done so that the data can be mapped, later, if you're into that.)

neigh_nums = {'1 Carondelet':'1', '2 Patch':'2', '3 Holly Hills':'3', '4 Boulevard Heights':'4', '5 Bevo Mill':'5', '6 Princeton Heights':'6', '7 South Hampton':'7', '8 St. Louis Hills':'8', '9 Lindenwood Park':'9', '10 Ellendale':'10', '11 Clifton Heights':'11', '12 The Hill':'12', '13 Southwest Garden':'13', '14 North Hampton':'14', '15 Tower Grove South':'15', '16 Dutchtown':'16', '17 Mount Pleasant':'17', '18 Marine Villa':'18', '19 Gravois Park':'19', '20 Kosciusko':'20', '21 Soulard':'21', '22 Benton Park':'22', '23 McKinley Heights':'23', '24 Fox Park':'24', '25 Tower Grove East':'25', '26 Compton Heights':'26', '27 Shaw':'27', '28 Botanical Heights':'28', '29 Tiffany': '29', '30 Benton Park West':'30', '31 The Gate District':'31', '32 Lafayette Square':'32',  '33 Peabody Darst Webbe':'33', '34 LaSalle Park':'34', '35 Downtown':'35', '36 Downtown West':'36', '37 Midtown':'37', '38 Central West End':'38', '39 Forest Park South East':'39', '40 Kings Oak':'40', '41 Cheltenham':'41', '42 Clayton-Tamm':'42', '43 Franz Park':'43', '44 Hi-Pointe':'44', '45 Wydown Skinker': '45', '46 Skinker DeBaliviere':'46', '47 DeBaliviere Place':'47', '48 West End':'48', '49 Visitation Park':'49', '50 Wells Goodfellow':'50', '51 Academy':'51', '52 Kingsway West':'52', '53 Fountain Park':'53', '54 Lewis Place':'54', '55 Kingsway East':'55', '56 Greater Ville':'56', '57 The Ville':'57', '58 Vandeventer':'58', '59 Jeff Vanderlou':'59', '60 St. Louis Place':'60', '61 Carr Square':'61', '62 Columbus Square':'62', '63 Old North St. Louis':'63', '64 Near North Riverfront':'64', '65 Hyde Park':'65', '66 College Hill':'66', '67 Fairground Neighborhood': '67', '68 O\'Fallon':'68', '69 Penrose':'69', '70 Mark Twain I-70 Industrial':'70', '71 Mark Twain':'71', '72 Walnut Park East':'72', '73 North Pointe':'73', '74 Baden':'74', '75 Riverview':'75', '76 Walnut Park West':'76', '77 Covenant Blu-Grand Center':'77', '78 Hamilton Heights':'78', '79 North Riverfront':'79', 'Carondelet Park':'80', 'Tower Grove Park':'81', 'Forest Park':'82', 'Fairgrounds Park':'83', 'Penrose Park':'84', 'O\'Fallon Park':'85', 'Willmore Park':'88', 'Aboussie Park':'89', 'Adams Park':'90', 'Alaska Park':'91', 'Aloe Plaza':'92', 'Benton Park':'93', 'Carnegie Playground':'94', 'Carondelet Lions Park':'95', 'Chain of Rocks Park':'96', 'Chouteau Park':'97', 'Christy Park':'98', 'DeSoto Park':'100', 'Dwight Davis Park':'101', 'Ernest J Russell Park': '102', 'Father Filipiac Park (UDAG)':'103', 'Fountain Park':'104', 'Freemont Park':'105', 'Garrison-Bratner Park':'106', 'Gravois Park':'107', 'Gwen Giles Park':'108', 'Handy Park':'109', 'Hyde Park':'110', 'Ivory Terry Park':'111', 'Jordan Chambers Park':'112', 'Joseph Dickman Park':'113', 'Lafayette Park':'114', 'Lucier Park':'115', 'Marie Fowler Park':'116', 'Marquette Park':'117', 'Minnesota & Hill Park':'118', 'Minnie Wood Park':'119', 'Mount Pleasant Park':'120', 'North Riverfront':'121', 'Ray Leisure Park':'122', 'Ruth Porter Mall':'123', 'Seay Park':'124', 'Sherman Park':'125', 'Sister Marie Charles Park':'126', 'Tandy Park':'128', 'Terry Park':'129', 'Tiffany Park':'130', 'Turner Playground':'131', 'Windsor Park':'132'}

In [55]:
# Mapping the neighborhood names to all the numbers in the data.

trash['Neighnum'] = trash['nhood'].map(neigh_nums)

In [56]:
# Here's a good place to break for a data export. Now, you can export and pivot trash complaints.

trash.to_csv('trash_for_pivot_10-6.27.22.csv')

In [57]:
# Now, let's look at response time.
# Import the file we just created.

response = pd.read_csv("trash_for_pivot_10-6.27.22.csv")

/Users/jayohday/.virtualenvs/trash/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (11,23,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [58]:
# Change the date-time columns to DATETIME format. This is important.

response['DATETIMEINIT'] = pd.to_datetime(response['DATETIMEINIT'])
response['DATETIMECLOSED'] = pd.to_datetime(response['DATETIMECLOSED'], errors='coerce')
response['DATEINVTDONE'] = pd.to_datetime(response['DATEINVTDONE'], errors='coerce')

In [59]:
# Creating a column to store the new variable, TIMEDIFF, for how long it takes to resolve a complaint.

response['TIMEDIFF'] = response['DATETIMECLOSED'] - response['DATETIMEINIT']

In [60]:
# Export the data here, if you like. It helps with checking any further analysis.

response.to_csv("data_with_timediff.csv")

In [61]:
# Okay, let's pivot this data by neighborhood, and complaint response time. 

datap = response.pivot(columns='nhood', values='TIMEDIFF')

In [62]:
# The next 79 lines set variables for each neighborhood median response time, for use in a later Python dictionary.

carondelet = datap[1].median()

patch = datap[2].median()

hollyh = datap[3].median()

bheights = datap[4].median()

bevo = datap[5].median()

pheights = datap[6].median()

shampton = datap[7].median()

stlhills = datap[8].median()

lpark = datap[9].median()

ellendale = datap[10].median()

cheights = datap[11].median()

hill = datap[12].median()

swgarden = datap[13].median()

nhampton = datap[14].median()

tgsouth = datap[15].median()

dutch = datap[16].median()

mpleasant = datap[17].median()

mvilla = datap[18].median()

gpark = datap[19].median()

kos = datap[20].median()

soulard = datap[21].median()

bpark = datap[22].median()

mckheights = datap[23].median()

foxpark = datap[24].median()

tgeast = datap[25].median()

compheights = datap[26].median()

shaw = datap[27].median()

botheights = datap[28].median()

tiff = datap[29].median()

bparkwest = datap[30].median()

gate = datap[31].median()

lafayette = datap[32].median() 

pdw = datap[33].median()

lasalle = datap[34].median()

dtown = datap[35].median()

dtownw = datap[36].median()

midtown = datap[37].median()

cwe = datap[38].median()

fparkse = datap[39].median()

kingsoak = datap[40].median()

chelt = datap[41].median()

claytamm = datap[42].median()

franzpark = datap[43].median()

hipoint = datap[44].median()

wydsk = datap[45].median()

skdeb = datap[46].median()

debplace = datap[47].median()

westend = datap[48].median()

vpark = datap[49].median()

wellsg = datap[50].median()

academy = datap[51].median()

kingsw = datap[52].median()

fountpark = datap[53].median()

lewplace = datap[54].median()

kingse = datap[55].median()

gville = datap[56].median()

ville = datap[57].median()

vand = datap[58].median()

jeffv = datap[59].median()

stlplace = datap[60].median()

carr = datap[61].median()

colsq = datap[62].median()

oldnstl = datap[63].median()

nnriv = datap[64].median()

hyde = datap[65].median()

college = datap[66].median()

fground = datap[67].median()

ofal = datap[68].median()

pen = datap[69].median()

mt = datap[71].median()

mt70 = datap[70].median()

wpeast = datap[72].median()

npoint = datap[73].median()

baden = datap[74].median()

riverv = datap[75].median()

wpwest = datap[76].median()

covblu = datap[77].median()

hamh = datap[78].median()

nriver = datap[79].median()

In [63]:
# Create a Python dictionary to store all medians and neighborhood names.
medians = {'Carondelet': carondelet, 'Patch': patch, 'Holly Hills': hollyh, 'Boulevard Heights': bheights, 'Bevo Mill': bevo, 'Princeton Heights': pheights, 'Southampton': shampton, 'St. Louis Hills': stlhills, 'Lindenwood Park': lpark, 'Ellendale': ellendale, 'Clifton Heights': cheights, 'The Hill': hill, 'Southwest Garden': swgarden, 'North Hampton': nhampton, 'Tower Grove South': tgsouth, 'Dutchtown': dutch, 'Mount Pleasant': mpleasant, 'Marine Villa': mvilla, 'Gravois Park': gpark, 'Kosciusko': kos, 'Soulard': soulard, 'Benton Park': bpark, 'McKinley Heights': mckheights, 'Fox Park': foxpark, 'Tower Grove East': tgeast, 'Compton Heights': compheights, 'Shaw': shaw, 'Botanical Heights': botheights, 'Tiffany': tiff, 'Benton Park West': bparkwest, 'Gate District': gate, 'Lafayette Square': lafayette, 'Peabody Darst Webbe': pdw, 'LaSalle': lasalle, 'Downtown': dtown, 'Downtown West': dtownw, 'Midtown': midtown, 'Central West End': cwe, 'Forest Park Southeast': fparkse, 'Kings Oak': kingsoak, 'Cheltenham': chelt, 'Clayton/Tamm': claytamm, 'Franz Park': franzpark, 'Hi-Point': hipoint, 'Wydown/Skinker': wydsk, 'Skinker/DeBaliviere': skdeb, 'DeBaliviere Place': debplace, 'West End': westend, 'Visitation Park': vpark, 'Wells/Goodfellow': wellsg, 'Academy': academy, 'Kingsway West': kingsw, 'Fountain Park': fountpark, 'Lewis Place': lewplace, 'Kingsway East': kingse, 'Greater Ville': gville, "The Ville": ville, 'Vandeventer': vand, 'JeffVanderLou': jeffv, 'St. Louis Place': stlplace, 'Carr Square': carr, 'Columbus Square': colsq, 'Old North STL': oldnstl, 'Near North Riverfront': nnriv, 'Hyde Park': hyde, 'College Hill': college, 'Fairground': fground, 'OFallon': ofal, 'Penrose': pen, 'Mark Twain I-70': mt70, 'Mark Twain': mt, 'Walnut Park East': wpeast, 'North Point': npoint, 'Baden': baden, 'Riverview': riverv, 'Walnut Park West': wpwest, 'Covenant Blu/Grand Center': covblu, 'Hamilton Heights': hamh, 'North Riverfront': nriver}

In [64]:
# Sort the dictionary you've created and labeled.

od = OrderedDict(sorted(medians.items(), key=lambda x:x[1], reverse=True))

In [65]:
# Print the median dictionary for 2010 through June 26, 2022.

pp.pprint(od)

OrderedDict([('The Ville', Timedelta('43 days 20:16:20.865500')),
             ('Hamilton Heights', Timedelta('42 days 08:16:18.097000')),
             ('Walnut Park East', Timedelta('41 days 03:25:45.537000')),
             ('Fairground', Timedelta('40 days 01:38:52.567000')),
             ('North Riverfront', Timedelta('39 days 09:34:38.310000')),
             ('Wells/Goodfellow', Timedelta('39 days 00:21:29.163000')),
             ('Hyde Park', Timedelta('33 days 16:07:28.317000')),
             ('College Hill', Timedelta('32 days 22:48:53.370000')),
             ('Academy', Timedelta('31 days 05:13:10.567000')),
             ('Walnut Park West', Timedelta('31 days 04:54:46.333500')),
             ('Greater Ville', Timedelta('31 days 04:43:49.813500')),
             ('Old North STL', Timedelta('30 days 12:04:52.083500')),
             ('Kingsway West', Timedelta('29 days 20:03:08.251500')),
             ('Fountain Park', Timedelta('28 days 19:31:36.540000')),
             ('Mark Twa